# Time series analysis on AWS
*Chapter 1 - Time series analysis overview*

## Initializations
---

In [ ]:
!pip install --quiet tqdm kaggle tsia ruptures

### Imports

In [ ]:
import matplotlib.colors as mpl_colors
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import ruptures as rpt
import sys
import tsia
import warnings
import zipfile

from matplotlib import gridspec
from sklearn.preprocessing import normalize
from tqdm import tqdm
from urllib.request import urlretrieve

### Parameters

In [ ]:
RAW_DATA = os.path.join('..', 'Data', 'raw')
DATA = os.path.join('..', 'Data')
warnings.filterwarnings("ignore")
os.makedirs(RAW_DATA, exist_ok=True)

%matplotlib inline
# plt.style.use('Solarize_Light2')
plt.style.use('fivethirtyeight')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.rcParams['figure.dpi'] = 300
plt.rcParams['lines.linewidth'] = 0.3
plt.rcParams['axes.titlesize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 4.5
plt.rcParams['ytick.labelsize'] = 4.5
plt.rcParams['grid.linewidth'] = 0.2
plt.rcParams['legend.fontsize'] = 5

### Helper functions

In [ ]:
def progress_report_hook(count, block_size, total_size):
    mb = int(count * block_size // 1048576)
    if count % 500 == 0:
        sys.stdout.write("\r{} MB downloaded".format(mb))
        sys.stdout.flush()

### Downloading datasets

#### **Dataset 1:** Household energy consumption

In [ ]:
ORIGINAL_DATA = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip'
ARCHIVE_PATH  = os.path.join(RAW_DATA, 'energy-consumption.zip')
FILE_NAME     = 'energy-consumption.csv'
FILE_PATH     = os.path.join(DATA, 'energy', FILE_NAME)
FILE_DIR      = os.path.dirname(FILE_PATH)

if not os.path.isfile(FILE_PATH):
    print("Downloading dataset (258MB), can take a few minutes depending on your connection")
    urlretrieve(ORIGINAL_DATA, ARCHIVE_PATH, reporthook=progress_report_hook)
    os.makedirs(os.path.join(DATA, 'energy'), exist_ok=True)

    print("\nExtracting data archive")
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall(FILE_DIR + '/')
    zip_ref.close()
    
    !rm -Rf $FILE_DIR/__MACOSX
    !mv $FILE_DIR/LD2011_2014.txt $FILE_PATH
    
else:
    print("File found, skipping download")

#### **Dataset 2:** Nasa Turbofan remaining useful lifetime

In [ ]:
ok = True
ok = ok and os.path.exists(os.path.join(DATA, 'turbofan', 'train_FD001.txt'))
ok = ok and os.path.exists(os.path.join(DATA, 'turbofan', 'test_FD001.txt'))
ok = ok and os.path.exists(os.path.join(DATA, 'turbofan', 'RUL_FD001.txt'))

if (ok):
    print("File found, skipping download")

else:
    print('Some datasets are missing, create working directories and download original dataset from the NASA repository.')
    
    # Making sure the directory already exists:
    os.makedirs(os.path.join(DATA, 'turbofan'), exist_ok=True)

    # Download the dataset from the NASA repository, unzip it and set
    # aside the first training file to work on:
    !wget https://ti.arc.nasa.gov/c/6/ --output-document=$RAW_DATA/CMAPSSData.zip
    !unzip $RAW_DATA/CMAPSSData.zip -d $RAW_DATA
    !cp $RAW_DATA/train_FD001.txt $DATA/turbofan/train_FD001.txt
    !cp $RAW_DATA/test_FD001.txt $DATA/turbofan/test_FD001.txt
    !cp $RAW_DATA/RUL_FD001.txt $DATA/turbofan/RUL_FD001.txt

#### **Dataset 3:** Human heartbeat

In [ ]:
ECG_DATA_SOURCE = 'http://www.timeseriesclassification.com/Downloads/ECG200.zip'
ARCHIVE_PATH  = os.path.join(RAW_DATA, 'ECG200.zip')
FILE_NAME     = 'ecg.csv'
FILE_PATH     = os.path.join(DATA, 'ecg', FILE_NAME)
FILE_DIR      = os.path.dirname(FILE_PATH)

if not os.path.isfile(FILE_PATH):
    urlretrieve(ECG_DATA_SOURCE, ARCHIVE_PATH)
    os.makedirs(os.path.join(DATA, 'ecg'), exist_ok=True)

    print("\nExtracting data archive")
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall(FILE_DIR + '/')
    zip_ref.close()
    
    !mv $DATA/ecg/ECG200_TRAIN.txt $FILE_PATH
    
else:
    print("File found, skipping download")

#### **Dataset 4:** Industrial pump data
To download this dataset from Kaggle, you will need to have an account and create a token that you install on your machine. You can follow [**this link**](https://www.kaggle.com/docs/api) to get started with the Kaggle API. Once generated, make sure your Kaggle token is stored in the `~/.kaggle/kaggle.json` file, or the next cells will issue an error. To get a Kaggle token, go to kaggle.com and create an account. Then navigate to **My account** and scroll down to the API section. There, click the **Create new API token** button:

<img src="../Assets/kaggle_api.png" />


In [ ]:
FILE_NAME    = 'pump-sensor-data.zip'
ARCHIVE_PATH = os.path.join(RAW_DATA, FILE_NAME)
FILE_PATH    = os.path.join(DATA, 'pump', 'sensor.csv')
FILE_DIR     = os.path.dirname(FILE_PATH)

if not os.path.isfile(FILE_PATH):
    if not os.path.exists('/home/ec2-user/.kaggle/kaggle.json'):
        os.makedirs('/home/ec2-user/.kaggle/', exist_ok=True)
        raise Exception('The kaggle.json token was not found.\nCreating the /home/ec2-user/.kaggle/ directory: put your kaggle.json file there once you have generated it from the Kaggle website')
    else:
        print('The kaggle.json token file was found: making sure it is not readable by other users on this system.')
        !chmod 600 /home/ec2-user/.kaggle/kaggle.json

    os.makedirs(os.path.join(DATA, 'pump'), exist_ok=True)
    !kaggle datasets download -d nphantawee/pump-sensor-data -p $RAW_DATA

    print("\nExtracting data archive")
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall(FILE_DIR + '/')
    zip_ref.close()
    
else:
    print("File found, skipping download")

#### **Dataset 5:** London household energy consumption with weather data

In [ ]:
FILE_NAME    = 'smart-meters-in-london.zip'
ARCHIVE_PATH = os.path.join(RAW_DATA, FILE_NAME)
FILE_PATH    = os.path.join(DATA, 'energy-london', 'smart-meters-in-london.zip')
FILE_DIR     = os.path.dirname(FILE_PATH)

# Checks if the data were already downloaded:
if os.path.exists(os.path.join(DATA, 'energy-london', 'acorn_details.csv')):
    print("File found, skipping download")
    
else:
    # Downloading and unzipping datasets from Kaggle:
    print("Downloading dataset (2.26G), can take a few minutes depending on your connection")
    os.makedirs(os.path.join(DATA, 'energy-london'), exist_ok=True)
    !kaggle datasets download -d jeanmidev/smart-meters-in-london -p $RAW_DATA
    
    print('Unzipping files...')
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall(FILE_DIR + '/')
    zip_ref.close()
    
    !rm $DATA/energy-london/*zip
    !rm $DATA/energy-london/*gz
    !mv $DATA/energy-london/halfhourly_dataset/halfhourly_dataset/* $DATA/energy-london/halfhourly_dataset
    !rm -Rf $DATA/energy-london/halfhourly_dataset/halfhourly_dataset
    !mv $DATA/energy-london/daily_dataset/daily_dataset/* $DATA/energy-london/daily_dataset
    !rm -Rf $DATA/energy-london/daily_dataset/daily_dataset

## Dataset visualization
---

### **1.** Household energy consumption

In [ ]:
%%time

FILE_PATH = os.path.join(DATA, 'energy', 'energy-consumption.csv')
energy_df = pd.read_csv(FILE_PATH, sep=';', decimal=',')
energy_df = energy_df.rename(columns={'Unnamed: 0': 'Timestamp'})
energy_df['Timestamp'] = pd.to_datetime(energy_df['Timestamp'])
energy_df = energy_df.set_index('Timestamp')
energy_df.iloc[100000:, 1:5].head()

In [ ]:
fig = plt.figure(figsize=(5, 1.876))
plt.plot(energy_df['MT_002'])
plt.title('Energy consumption for household MT_002')
plt.show()

### **2.** NASA Turbofan data

In [ ]:
FILE_PATH = os.path.join(DATA, 'turbofan', 'train_FD001.txt')
turbofan_df = pd.read_csv(FILE_PATH, header=None, sep=' ')
turbofan_df.dropna(axis='columns', how='all', inplace=True)
print('Shape:', turbofan_df.shape)
turbofan_df.head(5)

In [ ]:
columns = [
    'unit_number',
    'cycle',
    'setting_1',
    'setting_2',
    'setting_3',
] + ['sensor_{}'.format(s) for s in range(1,22)]
turbofan_df.columns = columns
turbofan_df.head()

In [ ]:
# Add a RUL column and group the data by unit_number:
turbofan_df['rul'] = 0
grouped_data = turbofan_df.groupby(by='unit_number')

# Loops through each unit number to get the lifecycle counts:
for unit, rul in enumerate(grouped_data.count()['cycle']):
    current_df = turbofan_df[turbofan_df['unit_number'] == (unit+1)].copy()
    current_df['rul'] = rul - current_df['cycle']
    turbofan_df[turbofan_df['unit_number'] == (unit+1)] = current_df

In [ ]:
df = turbofan_df.iloc[:, [0,1,2,3,4,5,6,25,26]].copy()
df = df[df['unit_number'] == 1]

def highlight_cols(s):
    return f'background-color: rgba(0, 143, 213, 0.3)'

df.head(10).style.applymap(highlight_cols, subset=['rul'])

### **3.** ECG Data

In [ ]:
FILE_PATH = os.path.join(DATA, 'ecg', 'ecg.csv')
ecg_df = pd.read_csv(FILE_PATH, header=None, sep='  ')
print('Shape:', ecg_df.shape)
ecg_df.head()

In [ ]:
plt.rcParams['lines.linewidth'] = 0.7
fig = plt.figure(figsize=(5,2))
label_normal = False
label_ischemia = False
for i in range(0,100):
    label = ecg_df.iloc[i, 0]
    if (label == -1):
        color = colors[1]
        
        if label_ischemia:
            plt.plot(ecg_df.iloc[i,1:96], color=color, alpha=0.5, linestyle='--', linewidth=0.5)
        else:
            plt.plot(ecg_df.iloc[i,1:96], color=color, alpha=0.5, label='Ischemia', linestyle='--')
            label_ischemia = True
            
    else:
        color = colors[0]
        
        if label_normal:
            plt.plot(ecg_df.iloc[i,1:96], color=color, alpha=0.5)
        else:
            plt.plot(ecg_df.iloc[i,1:96], color=color, alpha=0.5, label='Normal')
            label_normal = True
    
plt.title('Human heartbeat activity')
plt.legend(loc='upper right', ncol=2)
plt.show()

### **4.** Industrial pump data

In [ ]:
FILE_PATH = os.path.join(DATA, 'pump', 'sensor.csv')
pump_df = pd.read_csv(FILE_PATH, sep=',')
pump_df.drop(columns={'Unnamed: 0'}, inplace=True)
pump_df['timestamp'] = pd.to_datetime(pump_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
pump_df = pump_df.set_index('timestamp')

pump_df['machine_status'].replace(to_replace='NORMAL', value=np.nan, inplace=True)
pump_df['machine_status'].replace(to_replace='BROKEN', value=1, inplace=True)
pump_df['machine_status'].replace(to_replace='RECOVERING', value=1, inplace=True)

print('Shape:', pump_df.shape)
pump_df.head()

In [ ]:
file_structure_df = pump_df.iloc[:, 0:10].resample('5D').mean()

In [ ]:
plt.rcParams['hatch.linewidth'] = 0.5
plt.rcParams['lines.linewidth'] = 0.5

fig = plt.figure(figsize=(5,1))
ax1 = fig.add_subplot(1,1,1)
plot1 = ax1.plot(pump_df['sensor_00'], label='Healthy pump')

ax2 = ax1.twinx()
plot2 = ax2.fill_between(
    x=pump_df.index, 
    y1=0.0, 
    y2=pump_df['machine_status'], 
    color=colors[1], 
    linewidth=0.0,
    edgecolor='#000000',
    alpha=0.5, 
    hatch="//////", 
    label='Broken pump'
)
ax2.grid(False)
ax2.set_yticks([])

labels = [plot1[0].get_label(), plot2.get_label()]

plt.legend(handles=[plot1[0], plot2], labels=labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -.4))
plt.title('Industrial pump sensor data')
plt.show()

### **5.** London household energy consumption with weather data

We want to filter out households that are are subject to the dToU tariff and keep only the ones with a known ACORN (i.e. not in the ACORN-U group): this will allow us to better model future analysis by adding the Acorn detail informations (which by definitions, won't be available for the ACORN-U group).

In [ ]:
household_filename = os.path.join(DATA, 'energy-london', 'informations_households.csv')
household_df = pd.read_csv(household_filename)
household_df = household_df[(household_df['stdorToU'] == 'Std') & (household_df['Acorn'] == 'ACORN-E')]
print(household_df.shape)
household_df.head()

#### Associating households with they energy consumption data
Each household (with an ID starting by `MACxxxxx` in the table above) has its consumption data stored in a block file name `block_xx`. This file is also available from the `informations_household.csv` file extracted above. We have the association between `household_id` and `block_file`: we can open each of them and keep the consumption for the households of interest. All these data will be concatenated into an `energy_df` dataframe:

In [ ]:
%%time

household_ids = household_df['LCLid'].tolist()
consumption_file = os.path.join(DATA, 'energy-london', 'hourly_consumption.csv')
min_data_points = ((pd.to_datetime('2020-12-31') - pd.to_datetime('2020-01-01')).days + 1)*24*2

if os.path.exists(consumption_file):
    print('Half-hourly consumption file already exists, loading from disk...')
    energy_df = pd.read_csv(consumption_file)
    energy_df['timestamp'] = pd.to_datetime(energy_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    print('Done.')
    
else:
    print('Half-hourly consumption file not found. We need to generate it.')
    
    # We know have the block number we can use to open the right file:
    energy_df = pd.DataFrame()
    target_block_files = household_df['file'].unique().tolist()
    print('- {} block files to process: '.format(len(target_block_files)), end='')
    df_list = []
    for block_file in tqdm(target_block_files):
        # Reads the current block file:
        current_filename = os.path.join(DATA, 'energy-london', 'halfhourly_dataset', '{}.csv'.format(block_file))
        df = pd.read_csv(current_filename)
        
        # Set readable column names and adjust data types:
        df.columns = ['household_id', 'timestamp', 'energy']
        df = df.replace(to_replace='Null', value=0.0)
        df['energy'] = df['energy'].astype(np.float64)
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
        
        # We filter on the households sampled earlier:
        df_list.append(df[df['household_id'].isin(household_ids)].reset_index(drop=True))
    
    # Concatenate with the main dataframe:
    energy_df = pd.concat(df_list, axis='index', ignore_index=True)
    
    datapoints = energy_df.groupby(by='household_id').count()
    datapoints = datapoints[datapoints['timestamp'] < min_data_points]
    hhid_to_remove = datapoints.index.tolist()
    energy_df = energy_df[~energy_df['household_id'].isin(hhid_to_remove)]

    # Let's save this dataset to disk, we will use it from now on:
    print('Saving file to disk... ', end='')
    energy_df.to_csv(consumption_file, index=False)
    print('Done.')

In [ ]:
start = np.min(energy_df['timestamp'])
end = np.max(energy_df['timestamp'])
weather_filename = os.path.join(DATA, 'energy-london', 'weather_hourly_darksky.csv')

weather_df = pd.read_csv(weather_filename)
weather_df['time'] = pd.to_datetime(weather_df['time'], format='%Y-%m-%d %H:%M:%S')
weather_df = weather_df.drop(columns=['precipType', 'icon', 'summary'])
weather_df = weather_df.sort_values(by='time')
weather_df = weather_df.set_index('time')
weather_df = weather_df[start:end]

# Let's make sure we have one datapoint per hour to match 
# the frequency used for the household energy consumption data:
weather_df = weather_df.resample(rule='1H').mean()     # This will generate NaN values timestamp missing data
weather_df = weather_df.interpolate(method='linear')   # This will fill the missing values with the average 

print(weather_df.shape)
weather_df

In [ ]:
energy_df = energy_df.set_index(['household_id', 'timestamp'])
energy_df

In [ ]:
hhid = household_ids[2]
hh_energy = energy_df.loc[hhid, :]
start = '2012-07-01'
end = '2012-07-15'

fig = plt.figure(figsize=(5,1))
ax1 = fig.add_subplot(1,1,1)
plot2 = ax1.fill_between(
    x=weather_df.loc[start:end, 'temperature'].index, 
    y1=0.0, 
    y2=weather_df.loc[start:end, 'temperature'], 
    color=colors[1], 
    linewidth=0.0,
    edgecolor='#000000',
    alpha=0.25, 
    hatch="//////", 
    label='Temperature'
)
ax1.set_ylim((0,40))
ax1.grid(False)

ax2 = ax1.twinx()
ax2.plot(hh_energy[start:end], label='Energy consumption', linewidth=2, color='#FFFFFF', alpha=0.5)
plot1 = ax2.plot(hh_energy[start:end], label='Energy consumption', linewidth=0.7)
ax2.set_title(f'Energy consumption for household {hhid}')

labels = [plot1[0].get_label(), plot2.get_label()]
plt.legend(handles=[plot1[0], plot2], labels=labels, loc='upper left', fontsize=3, ncol=2)

plt.show()

In [ ]:
acorn_filename = os.path.join(DATA, 'energy-london', 'acorn_details.csv')
acorn_df = pd.read_csv(acorn_filename, encoding='ISO-8859-1')
acorn_df = acorn_df.sample(10).loc[:, ['MAIN CATEGORIES', 'CATEGORIES', 'REFERENCE', 'ACORN-A', 'ACORN-B', 'ACORN-E']]
acorn_df

## File structure exploration
---

In [ ]:
from IPython.display import display_html

def display_multiple_dataframe(*args, max_rows=None, max_cols=None):
    html_str = ''
    for df in args:
        html_str += df.to_html(max_cols=max_cols, max_rows=max_rows)
        
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)

In [ ]:
display_multiple_dataframe(
    file_structure_df[['sensor_00']],
    file_structure_df[['sensor_01']],
    file_structure_df[['sensor_03']],
    max_rows=10, max_cols=None
)

In [ ]:
display_multiple_dataframe(
    file_structure_df.loc['2018-04', :].head(6),
    file_structure_df.loc['2018-05', :].head(6),
    file_structure_df.loc['2018-06', :].head(6),
    max_rows=None, max_cols=2
)

In [ ]:
display_multiple_dataframe(
    file_structure_df.loc['2018-04', ['sensor_00']].head(6),
    file_structure_df.loc['2018-05', ['sensor_00']].head(6),
    file_structure_df.loc['2018-06', ['sensor_00']].head(6),
    max_rows=10, max_cols=None
)
display_multiple_dataframe(
    file_structure_df.loc['2018-04', ['sensor_01']].head(6),
    file_structure_df.loc['2018-05', ['sensor_01']].head(6),
    file_structure_df.loc['2018-06', ['sensor_01']].head(6),
    max_rows=10, max_cols=None
)
print('.\n.\n.')
display_multiple_dataframe(
    file_structure_df.loc['2018-04', ['sensor_09']].head(6),
    file_structure_df.loc['2018-05', ['sensor_09']].head(6),
    file_structure_df.loc['2018-06', ['sensor_09']].head(6),
    max_rows=10, max_cols=None
)

In [ ]:
df1 = pump_df.iloc[:, [0]].resample('5D').mean()
df2 = pump_df.iloc[:, [1]].resample('2D').mean()
df3 = pump_df.iloc[:, [2]].resample('7D').mean()

display_multiple_dataframe(
    df1.head(10), df2.head(10), df3.head(10),
    pd.merge(pd.merge(df1, df2, left_index=True, right_index=True, how='outer'), df3, left_index=True, right_index=True, how='outer').head(10),
    max_rows=None, max_cols=None
)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.merge(pd.merge(df1, df2, left_index=True, right_index=True, how='outer'), df3, left_index=True, right_index=True, how='outer').head(10)

In [ ]:
plt.figure(figsize=(5,1))
for i in range(len(colors)):
    plt.plot(file_structure_df[f'sensor_0{i}'], linewidth=2, alpha=0.5, label=colors[i])

plt.legend()
plt.show()

## Visualization
---

In [ ]:
fig = plt.figure(figsize=(5,1))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

plot_sensor_0 = ax1.plot(pump_df['sensor_00'], label='Sensor 0', color=colors[0], linewidth=1, alpha=0.8)
plot_sensor_1 = ax2.plot(pump_df['sensor_01'], label='Sensor 1', color=colors[1], linewidth=1, alpha=0.8)
ax2.grid(False)
plt.title('Pump sensor values (2 sensors)')
plt.legend(handles=[plot_sensor_0[0], plot_sensor_1[0]], ncol=2, loc='lower right')
plt.show()

In [ ]:
reduced_pump_df = pump_df.loc[:, 'sensor_00':'sensor_14']
reduced_pump_df = reduced_pump_df.replace([np.inf, -np.inf], np.nan)
reduced_pump_df = reduced_pump_df.fillna(0.0)
reduced_pump_df = reduced_pump_df.astype(np.float32)
scaled_pump_df = pd.DataFrame(normalize(reduced_pump_df), index=reduced_pump_df.index, columns=reduced_pump_df.columns)
scaled_pump_df

In [ ]:
fig = plt.figure(figsize=(5,1))

for i in range(0,15):
    plt.plot(scaled_pump_df.iloc[:, i], alpha=0.6)

plt.title('Pump sensor values (15 sensors)')
plt.show()

In [ ]:
pump_df2 = pump_df.copy()

pump_df2 = pump_df2.replace([np.inf, -np.inf], np.nan)
pump_df2 = pump_df2.fillna(0.0)
pump_df2 = pump_df2.astype(np.float32)

pump_description = pump_df2.describe().T
constant_signals = pump_description[pump_description['min'] == pump_description['max']].index.tolist()
pump_df2 = pump_df2.drop(columns=constant_signals)

features = pump_df2.columns.tolist()

In [ ]:
def hex_to_rgb(hex_color):
    """
    Converts a color string in hexadecimal format to RGB format.
    
    PARAMS
    ======
        hex_color: string
            A string describing the color to convert from hexadecimal. It can
            include the leading # character or not
    
    RETURNS
    =======
        rgb_color: tuple
            Each color component of the returned tuple will be a float value
            between 0.0 and 1.0
    """
    hex_color = hex_color.lstrip('#')
    rgb_color = tuple(int(hex_color[i:i+2], base=16) / 255.0 for i in [0, 2, 4])
    return rgb_color

def plot_timeseries_strip_chart(binned_timeseries, signal_list, fig_width=12, signal_height=0.15, dates=None, day_interval=7):
    # Build a suitable colormap:
    colors_list = [
        hex_to_rgb('#DC322F'), 
        hex_to_rgb('#B58900'), 
        hex_to_rgb('#2AA198')
    ]
    cm = mpl_colors.LinearSegmentedColormap.from_list('RdAmGr', colors_list, N=len(colors_list))
    
    fig = plt.figure(figsize=(fig_width, signal_height * binned_timeseries.shape[0]))
    ax = fig.add_subplot(1,1,1)
    
    # Devising the extent of the actual plot:
    if dates is not None:
        dnum = mdates.date2num(dates)
        start = dnum[0] - (dnum[1]-dnum[0])/2.
        stop = dnum[-1] + (dnum[1]-dnum[0])/2.
        extent = [start, stop, 0, signal_height * (binned_timeseries.shape[0])]
        
    else:
        extent = None
        
    # Plot the matrix:
    im = ax.imshow(binned_timeseries, 
                   extent=extent, 
                   aspect="auto", 
                   cmap=cm, 
                   origin='lower')
    
    # Adjusting the x-axis if we provide dates:
    if dates is not None:
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        for tick in ax.xaxis.get_major_ticks():
            tick.label.set_fontsize(4)
            tick.label.set_rotation(60)
            tick.label.set_fontweight('bold')

        ax.tick_params(axis='x', which='major', pad=7, labelcolor='#000000')
        plt.xticks(ha='right')
        
    # Adjusting the y-axis:
    ax.yaxis.set_major_locator(ticker.MultipleLocator(signal_height))
    ax.set_yticklabels(signal_list, verticalalignment='bottom', fontsize=4)
    ax.set_yticks(np.arange(len(signal_list)) * signal_height)

    plt.grid()
    return ax

In [ ]:
from IPython.display import display, Markdown, Latex

# Build a list of dataframes, one per sensor:
df_list = []
for f in features[:1]:
    df_list.append(pump_df2[[f]])

# Discretize each signal in 3 bins:
array = tsia.markov.discretize_multivariate(df_list)

fig = plt.figure(figsize=(5.5, 0.6))
plt.plot(pump_df2['sensor_00'], linewidth=0.7, alpha=0.6)
plt.title('Line plot of the pump sensor 0')
plt.show()

display(Markdown('<img src="arrow.png" align="left" style="padding-left: 730px"/>'))


# Plot the strip chart:
ax = plot_timeseries_strip_chart(
    array, 
    signal_list=features[:1],
    fig_width=5.21,
    signal_height=0.2,
    dates=df_list[0].index.to_pydatetime(),
    day_interval=2
)
ax.set_title('Strip chart of the pump sensor 0');

In [ ]:
# Build a list of dataframes, one per sensor:
df_list = []
for f in features:
    df_list.append(pump_df2[[f]])

# Discretize each signal in 3 bins:
array = tsia.markov.discretize_multivariate(df_list)

# Plot the strip chart:
fig = plot_timeseries_strip_chart(
    array, 
    signal_list=features,
    fig_width=5.5,
    signal_height=0.1,
    dates=df_list[0].index.to_pydatetime(),
    day_interval=2
)

### Recurrence plot

In [ ]:
from pyts.image import RecurrencePlot
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField

In [ ]:
hhid = household_ids[2]
hh_energy = energy_df.loc[hhid, :]
pump_extract_df = pump_df.iloc[:800, 0].copy()

rp = RecurrencePlot(threshold='point', percentage=30)
weather_rp = rp.fit_transform(weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.reshape(1, -1))
energy_rp = rp.fit_transform(hh_energy['2012-07-01':'2012-07-15'].values.reshape(1, -1))
pump_rp = rp.fit_transform(pump_extract_df.values.reshape(1, -1))


fig = plt.figure(figsize=(5.5, 2.4))
gs = gridspec.GridSpec(nrows=3, ncols=2, width_ratios=[3,1], hspace=0.8, wspace=0.0)

# Pump sensor 0:
ax = fig.add_subplot(gs[0])
ax.plot(pump_extract_df, label='Pump sensor 0')
ax.set_title(f'Pump sensor 0')

ax = fig.add_subplot(gs[1])
ax.imshow(pump_rp[0], cmap='binary', origin='lower')
ax.axis('off')

# Energy consumption line plot and recurrence plot:
ax = fig.add_subplot(gs[2])
plot1 = ax.plot(hh_energy['2012-07-01':'2012-07-15'], color=colors[1])
ax.set_title(f'Energy consumption for household {hhid}')

ax = fig.add_subplot(gs[3])
ax.imshow(energy_rp[0], cmap='binary', origin='lower')
ax.axis('off')

# Daily temperature line plot and recurrence plot:
ax = fig.add_subplot(gs[4])
start = '2012-07-01'
end = '2012-07-15'
ax.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2])
ax.set_title(f'Daily temperature')

ax = fig.add_subplot(gs[5])
ax.imshow(weather_rp[0], cmap='binary', origin='lower')
ax.axis('off')

plt.show()

In [ ]:
hhid = household_ids[2]
hh_energy = energy_df.loc[hhid, :]
pump_extract_df = pump_df.iloc[:800, 0].copy()

gaf = GramianAngularField(image_size=48, method='summation')
weather_gasf = gaf.fit_transform(weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.reshape(1, -1))
energy_gasf = gaf.fit_transform(hh_energy['2012-07-01':'2012-07-15'].values.reshape(1, -1))
pump_gasf = gaf.fit_transform(pump_extract_df.values.reshape(1, -1))

fig = plt.figure(figsize=(5.5, 2.4))
gs = gridspec.GridSpec(nrows=3, ncols=2, width_ratios=[3,1], hspace=0.8, wspace=0.0)

# Pump sensor 0:
ax = fig.add_subplot(gs[0])
ax.plot(pump_extract_df, label='Pump sensor 0')
ax.set_title(f'Pump sensor 0')

ax = fig.add_subplot(gs[1])
ax.imshow(pump_gasf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

# Energy consumption line plot and recurrence plot:
ax = fig.add_subplot(gs[2])
plot1 = ax.plot(hh_energy['2012-07-01':'2012-07-15'], color=colors[1])
ax.set_title(f'Energy consumption for household {hhid}')

ax = fig.add_subplot(gs[3])
ax.imshow(energy_gasf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

# Daily temperature line plot and recurrence plot:
ax = fig.add_subplot(gs[4])
start = '2012-07-01'
end = '2012-07-15'
ax.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2])
ax.set_title(f'Daily temperature')

ax = fig.add_subplot(gs[5])
ax.imshow(weather_gasf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

plt.show()

In [ ]:
mtf = MarkovTransitionField(image_size=48)

weather_mtf = mtf.fit_transform(weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.reshape(1, -1))
energy_mtf = mtf.fit_transform(hh_energy['2012-07-01':'2012-07-15'].values.reshape(1, -1))
pump_mtf = mtf.fit_transform(pump_extract_df.values.reshape(1, -1))

fig = plt.figure(figsize=(5.5, 2.4))
gs = gridspec.GridSpec(nrows=3, ncols=2, width_ratios=[3,1], hspace=0.8, wspace=0.0)

# Pump sensor 0:
ax = fig.add_subplot(gs[0])
ax.plot(pump_extract_df, label='Pump sensor 0')
ax.set_title(f'Pump sensor 0')

ax = fig.add_subplot(gs[1])
ax.imshow(pump_mtf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

# Energy consumption line plot and recurrence plot:
ax = fig.add_subplot(gs[2])
plot1 = ax.plot(hh_energy['2012-07-01':'2012-07-15'], color=colors[1])
ax.set_title(f'Energy consumption for household {hhid}')

ax = fig.add_subplot(gs[3])
ax.imshow(energy_mtf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

# Daily temperature line plot and recurrence plot:
ax = fig.add_subplot(gs[4])
start = '2012-07-01'
end = '2012-07-15'
ax.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2])
ax.set_title(f'Daily temperature')

ax = fig.add_subplot(gs[5])
ax.imshow(weather_mtf[0], cmap='RdBu_r', origin='lower')
ax.axis('off')

plt.show()

In [ ]:
import matplotlib
import matplotlib.cm as cm
import networkx as nx
import community

def compute_network_graph(markov_field):
    G = nx.from_numpy_matrix(markov_field[0])

    # Uncover the communities in the current graph:
    communities = community.best_partition(G)
    nb_communities = len(pd.Series(communities).unique())
    cmap = 'autumn'

    # Compute node colors and edges colors for the modularity encoding:
    edge_colors = [matplotlib.colors.to_hex(cm.get_cmap(cmap)(communities.get(v)/(nb_communities - 1))) for u,v in G.edges()]
    node_colors = [communities.get(node) for node in G.nodes()]
    node_size = [nx.average_clustering(G, [node])*90 for node in G.nodes()]

    # Builds the options set to draw the network graph in the "modularity" configuration:
    options = {
        'node_size': 10,
        'edge_color': edge_colors,
        'node_color': node_colors,
        'linewidths': 0,
        'width': 0.1,
        'alpha': 0.6,
        'with_labels': False,
        'cmap': cmap
    }
    
    return G, options

In [ ]:
fig = plt.figure(figsize=(5.5, 2.4))
gs = gridspec.GridSpec(nrows=3, ncols=2, width_ratios=[3,1], hspace=0.8, wspace=0.0)

# Pump sensor 0:
ax = fig.add_subplot(gs[0])
ax.plot(pump_extract_df, label='Pump sensor 0')
ax.set_title(f'Pump sensor 0')

ax = fig.add_subplot(gs[1])
G, options = compute_network_graph(weather_mtf)
nx.draw_networkx(G, **options, pos=nx.spring_layout(G), ax=ax)
ax.axis('off')

# Energy consumption line plot and recurrence plot:
ax = fig.add_subplot(gs[2])
plot1 = ax.plot(hh_energy['2012-07-01':'2012-07-15'], color=colors[1])
ax.set_title(f'Energy consumption for household {hhid}')

ax = fig.add_subplot(gs[3])
G, options = compute_network_graph(energy_mtf)
nx.draw_networkx(G, **options, pos=nx.spring_layout(G), ax=ax)
ax.axis('off')

# Daily temperature line plot and recurrence plot:
ax = fig.add_subplot(gs[4])
start = '2012-07-01'
end = '2012-07-15'
ax.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2])
ax.set_title(f'Daily temperature')

ax = fig.add_subplot(gs[5])
G, options = compute_network_graph(weather_mtf)
nx.draw_networkx(G, **options, pos=nx.spring_layout(G), ax=ax)
ax.axis('off')

plt.show()

## Symbolic representation
---

In [ ]:
from pyts.bag_of_words import BagOfWords

window_size, word_size = 30, 5
bow = BagOfWords(window_size=window_size, word_size=word_size, window_step=window_size, numerosity_reduction=False)
X = weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.reshape(1, -1)
X_bow = bow.transform(X)
time_index = weather_df.loc['2013-01-01':'2013-01-31']['temperature'].index
len(X_bow[0].replace(' ', ''))

In [ ]:
# Plot the considered subseries
plt.figure(figsize=(5, 2))
splits_series = np.linspace(0, X.shape[1], 1 + X.shape[1] // window_size, dtype='int64')
for start, end in zip(splits_series[:-1], np.clip(splits_series[1:] + 1, 0, X.shape[1])):
    plt.plot(np.arange(start, end), X[0, start:end], 'o-', linewidth=0.5, ms=0.1)

# Plot the corresponding letters
splits_letters = np.linspace(0, X.shape[1], 1 + word_size * X.shape[1] // window_size)
splits_letters = ((splits_letters[:-1] + splits_letters[1:]) / 2)
splits_letters = splits_letters.astype('int64')

for i, (x, text) in enumerate(zip(splits_letters, X_bow[0].replace(' ', ''))):
    t = plt.text(x, X[0, x], text, color="C{}".format(i // 5), fontsize=3.5)
    t.set_bbox(dict(facecolor='#FFFFFF', alpha=0.5, edgecolor="C{}".format(i // 5), boxstyle='round4'))

plt.title('Bag-of-words representation for weather temperature')
plt.tight_layout()
plt.show()

In [ ]:
from pyts.transformation import WEASEL
from sklearn.preprocessing import LabelEncoder

In [ ]:
X_train = ecg_df.iloc[:, 1:].values
y_train = ecg_df.iloc[:, 0]
y_train = LabelEncoder().fit_transform(y_train)
weasel = WEASEL(word_size=3, n_bins=3, window_sizes=[10, 25], sparse=False)
X_weasel = weasel.fit_transform(X_train, y_train)
vocabulary_length = len(weasel.vocabulary_)

In [ ]:
plt.figure(figsize=(5,1.5))
width = 0.4
x = np.arange(vocabulary_length) - width / 2
for i in range(len(X_weasel[y_train == 0])):
    if i == 0:
        plt.bar(x, X_weasel[y_train == 0][i], width=width, alpha=0.25, color=colors[1], label='Time series for Ischemia')
    else:
        plt.bar(x, X_weasel[y_train == 0][i], width=width, alpha=0.25, color=colors[1])
    
for i in range(len(X_weasel[y_train == 1])):
    if i == 0:
        plt.bar(x+width, X_weasel[y_train == 1][i], width=width, alpha=0.25, color=colors[0], label='Time series for Normal heartbeat')
    else:
        plt.bar(x+width, X_weasel[y_train == 1][i], width=width, alpha=0.25, color=colors[0])
        
plt.xticks(
    np.arange(vocabulary_length),
    np.vectorize(weasel.vocabulary_.get)(np.arange(X_weasel[0].size)),
    fontsize=2,
    rotation=60
)
    
plt.legend(loc='upper right')
plt.show()

## Statistics
---

In [ ]:
plt.rcParams['xtick.labelsize'] = 3

import statsmodels.api as sm

fig = plt.figure(figsize=(5.5, 3))
gs = gridspec.GridSpec(nrows=3, ncols=2, width_ratios=[1,1], hspace=0.8)

# Pump
ax = fig.add_subplot(gs[0])
ax.plot(pump_extract_df, label='Pump sensor 0')
ax.set_title(f'Pump sensor 0')
ax.tick_params(axis='x', which='both', labelbottom=False)

ax = fig.add_subplot(gs[1])
sm.graphics.tsa.plot_acf(pump_extract_df.values.squeeze(), ax=ax, markersize=1, title='')
ax.set_ylim(-1.2, 1.2)
ax.tick_params(axis='x', which='major', labelsize=4)

# Energy consumption
ax = fig.add_subplot(gs[2])
ax.plot(hh_energy['2012-07-01':'2012-07-15'], color=colors[1])
ax.set_title(f'Energy consumption for household {hhid}')
ax.tick_params(axis='x', which='both', labelbottom=False)

ax = fig.add_subplot(gs[3])
sm.graphics.tsa.plot_acf(hh_energy['2012-07-01':'2012-07-15'].values.squeeze(), ax=ax, markersize=1, title='')
ax.set_ylim(-0.3, 0.3)
ax.tick_params(axis='x', which='major', labelsize=4)

# Daily temperature:
ax = fig.add_subplot(gs[4])
start = '2012-07-01'
end = '2012-07-15'
ax.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2])
ax.set_title(f'Daily temperature')
ax.tick_params(axis='x', which='both', labelbottom=False)

ax = fig.add_subplot(gs[5])
sm.graphics.tsa.plot_acf(weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.squeeze(), ax=ax, markersize=1, title='')
ax.set_ylim(-1.2, 1.2)
ax.tick_params(axis='x', which='major', labelsize=4)

plt.show()

In [ ]:
from statsmodels.tsa.seasonal import STL

endog = endog.resample('30T').mean()

In [ ]:
plt.rcParams['lines.markersize'] = 1

title = f'Energy consumption for household {hhid}'
endog = hh_energy['2012-07-01':'2012-07-15']
endog.columns = [title]
endog = endog[title]
stl = STL(endog, period=48)
res = stl.fit()
fig = res.plot()

fig = plt.gcf()
fig.set_size_inches(5.5, 4)

plt.show()

## Binary segmentation
---

In [ ]:
signal = weather_df.loc['2013-01-01':'2013-01-31']['temperature'].values.squeeze()
algo = rpt.Binseg(model='l2').fit(signal)
my_bkps = algo.predict(n_bkps=3)

In [ ]:
my_bkps = [0] + my_bkps
my_bkps

In [ ]:
fig = plt.figure(figsize=(5.5,1))
start = '2012-07-01'
end = '2012-07-15'
plt.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color='#FFFFFF', linewidth=1.2, alpha=0.8)
plt.plot(weather_df.loc['2013-01-01':'2013-01-31']['temperature'], color=colors[2], linewidth=0.7)

plt.title(f'Daily temperature')
plt.xticks(rotation=60, fontsize=4)

weather_index = weather_df.loc['2013-01-01':'2013-01-31']['temperature'].index

for index, bkps in enumerate(my_bkps[:-1]):
    x1 = weather_index[my_bkps[index]]
    x2 = weather_index[np.clip(my_bkps[index+1], 0, len(weather_index)-1)]
    
    plt.axvspan(x1, x2, color=colors[index % 5], alpha=0.2)

plt.title('Daily temperature segmentation')
plt.show()